In [1]:
naver_news_section_dict: dict[str, int] = {
    '정치': 100,
    '경제': 101,
    '사회': 102,
    '생활': 103,
    '문화': 103,
    'IT': 104,
    '과학': 104,
    '세계': 105
}

naver_news_home_url = 'https://news.naver.com/'

In [2]:
query_section = '경제'
section = naver_news_section_dict[query_section]
section_url = naver_news_home_url + 'section/{}'.format(section)

In [3]:
section_url

'https://news.naver.com/section/101'

In [4]:
import requests
from bs4 import BeautifulSoup

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
}
response = requests.get(section_url, headers = headers)
soup = BeautifulSoup(response.text, 'html.parser')

headlines = []

for element in soup.select('.sa_text_strong'):
    headlines.append(element.text)

headlines

[' 맞춤형 여행 일정에 맛집 추천까지…삼성 비전 AI에 "와우"[CES 2025]',
 '이복현, 최상목 또 공개 지지… 尹 체포 불응엔 "법 집행, 대통령도 영향 미쳐선 안돼"',
 '삼성, ‘2025 삼성 명장’ 15명 공개…사상 최대',
 '인사·예산권 박탈에 연임 불가…새마을금고 중앙회장 힘 뺐다',
 '정의선 “현재 대형 위기…도전 못 피해가”',
 '용역주고 서면 늑장 발급…넥슨·크래프톤에 과징금',
 'AI가 일상 곳곳으로...SK ‘CES 2025’에서 미래 기술 선보인다',
 "'60억 못 막아' 63빌딩 지은 '신동아건설' 법정관리 신청(종합)",
 '거위털 80%라면서 실제는 30%… ‘후아유’도 함량 미달',
 'KB국민은행, 둔촌주공 잔금대출 한도 3000억 증액… 총 6000억 공급',
 '[단독] “이봐, 문제는 수익률이야”…요즘 기업 주총장서 가장 목소리 크다는 이 큰손',
 '“입주 앞두고 웬 날벼락”…중견 건설사 줄도산 공포에 계약자 피해 우려',
 '‘63빌딩 시공사’ 신동아건설 법정관리 신청…수도권 사업장 ‘비상’',
 'MBK "이그니오 팔아 100배 수익 누구"…고려아연 "이해 부족"(종합)',
 "증시 돌아온 개미들... 대기자금 60兆 육박 [새해 '머니무브' 조짐]",
 '악재 넘치는데 목표가 2배? 증권사 뻥튀기 남발',
 '"반등장 올라타자" 무르익는 기대… 내일 삼성 실적 등 변수 [새해 \'머니무브\' 조짐]',
 '"AI 청사진 제시"… CES 현장 누비는 최태원, 美 빅테크들과 AI협력 폭 넓힌다 [CES 2025 개막]',
 '샘 올트먼 “인간 수준의 AGI, 트럼프 임기 중 개발될 것”',
 '美로 몰리는 돈… "트럼프 2.0 시대, 달러의 경쟁자는 없다"  [강달러에 흔들리는 아시아 경제 ]',
 '정의선 "위기를 기회로… 고객 생각하면 답 나온다"',
 '"집값 하락기에 규제 강화…실수요자·세입자 자금마련 문턱 높아질 것"',
 '"거위털 80% 패딩 알고보니 30%"…패딩충전재 혼용률 논란',


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime

def extract_news(section='경제') -> pd.DataFrame:

    # CONSTANT --------------------------------------------------------------------------------------------------
    # 네이버 Section과 url상에서 사용하는 id
    naver_news_section_dict: dict[str, int] = {
        '정치': 100,
        '경제': 101,
        '사회': 102,
        '생활': 103,
        '문화': 103,
        'IT': 104,
        '과학': 104,
        '세계': 105
    }

    # 해당 section의 url
    section_url = 'https://news.naver.com/section/{}'.format(naver_news_section_dict[section])

    # USER_AGENT -----------------------------------------------------------------------------------------------
    # mac용 chrome 헤더
    header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
    }
    # window용 chrome 헤더
    '''
    '''


    response = requests.get(section_url, headers = header)
    soup = BeautifulSoup(response.text, 'html.parser')
    headlines = []
    news_url = []
    news_main_all = []
    for element in soup.select('.sa_text_title'):
        headlines.append(element.text.strip())
        
        cur_url = element['data-imp-url']
        news_url.append(cur_url)

        response2 = requests.get(cur_url, headers=header)
        soup_for_main = BeautifulSoup(response2.text, 'html.parser')
        main_raw = soup_for_main.select('#dic_area')
        main_tmp_list = []
        m_txt = ''
        for m in main_raw:
            m_text = m.text
            m_text = m_text.strip()
            main_tmp_list.append(m_text)
        main_string = ''.join(main_tmp_list)

        news_main_all.append(main_string)

    df = pd.DataFrame({
        '헤드라인': headlines,
        'URL': news_url,
        '본문': news_main_all
    })
    directory = os.path.join('./', 'scrapped')
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_name = '{}-{}.csv'.format(section, datetime.today().strftime("%Y-%m-%d"))
    df.to_csv(os.path.join(directory, file_name), index=False, encoding="utf-8-sig")
    return df
    

In [13]:
economy_section = extract_news()
economy_section

,헤드라인,URL,본문
0,"'시공 순위 58위' 신동아건설, 법원에 기업회생절차 신청",https://n.news.naver.com/mnews/article/214/000...,[신동아건설 제공]중견건설사인 신동아건설이 오늘 서울회생법원에 기업 회생절차를 신청...
1,김정수 삼양식품 부회장 “올해 사업 다각화…헬스케어 사업 확대”[신년사],https://n.news.naver.com/mnews/article/366/000...,잭앤펄스 팝업스토어를 둘러보는 김정수 삼양라운드스퀘어 부회장/삼양라운드스퀘어 제공 ...
2,"공정위, '하도급법 위반' 크래프톤·넥슨·엔씨 제재",https://n.news.naver.com/mnews/article/422/000...,국내 주요 게임업체들이 하청을 주고도 계약서를 제대로 내주지 않은 혐의로 공정거래위...
3,"SK, CES서 글로벌 AI 협력모델 만든다…최태원 선봉장",https://n.news.naver.com/mnews/article/079/000...,"[CES2025]'Innovative AI, Sustainable Tomorrow'..."
4,"""불법 공매도 적발"" …거래도 공매도전산시스템 개발 마쳐",https://n.news.naver.com/mnews/article/654/000...,"거래소, 3월부터 모의시장 운영\n\n\n\n▲ 공매도 중앙점검시스템. 한국거래소 ..."
5,"삼성·LG, ‘홈 AI’ 혁신의 길 열다",https://n.news.naver.com/mnews/article/022/000...,라스베이거스서 ‘CES 2025’ 개막글로벌 혁신 리더들이 최신 기술력과 미래 비전...
6,"최 권한대행 ""가용 수단 총동원해 미 신정부와 소통""",https://n.news.naver.com/mnews/article/374/000...,[최상목 대통령 권한대행 부총리 겸 기획재정부 장관이 2일 정부서울청사에서 열린 경...
7,"정의선 ""퍼펙트스톰 속에서 기회 찾자""",https://n.news.naver.com/mnews/article/015/000...,"현대차그룹 신년회서 도전·혁신 강조美中 갈등·전기차 캐즘 등 위기 돌파 의지""불확실..."
8,“삼성맨이라면 꿈꾸는 롤모델”...올해 최고의 기술 인재로 꼽힌 15명은?,https://n.news.naver.com/mnews/article/009/000...,"2025 삼성 명장. [사진출처 = 삼성전자]삼성은 제조, 설비, 품질, 인프라, ..."
9,오세훈 지원사격 나선 SH…미리내집 전담부서 신설·한강사업단 사장 직속,https://n.news.naver.com/mnews/article/011/000...,오세훈 표 저출생 극복 방안 지원 나선 SH장기전세주택 전담 부서 신설한강사업단도 ...


In [56]:
social_section = extract_news('사회')
social_section

,헤드라인,URL
0,화요일 아침 서울 체감 '-11도'…맹추위에 충남·호남 대설,https://n.news.naver.com/mnews/article/001/001...
1,"""경찰, 공수처 '체포 일임' 공문 법적 결함 있다 판단…따르기 힘들다고""",https://n.news.naver.com/mnews/article/437/000...
2,"[속보] 경찰 ""경호처, 체포저지선 구축에 경호부대 사병 동원 확인""",https://n.news.naver.com/mnews/article/011/000...
3,"공수처 ""공조수사본부 체제하에 경찰과 尹체포영장 집행 만전""",https://n.news.naver.com/mnews/article/001/001...
4,"경찰 ""체포영장 집행 방해하는 경호처 체포 방안 검토""",https://n.news.naver.com/mnews/article/057/000...
5,"[단독] 박안수 지시에 계엄사령부 34명 구성…""해제 뒤 용산 출발""",https://n.news.naver.com/mnews/article/057/000...
6,"검찰, '실탄 지시' 문상호 정보사령관 구속 기소(2보)",https://n.news.naver.com/mnews/article/421/000...
7,"[속보] 공조본 ""尹 체포영장 기한 연장 위해 영장 재청구""",https://n.news.naver.com/mnews/article/655/000...
8,"헌재 ""'내란죄' 탄핵사유 철회 권유 안 해""",https://n.news.naver.com/mnews/article/031/000...
9,‘존폐 논란’ 차고지증명제 ‘유지’…“적용 대상 완화·불편 개선”,https://n.news.naver.com/mnews/article/056/001...


In [7]:
from datetime import datetime

datetime.today().strftime("%Y-%m-%d")

'2025-01-06'

In [ ]:
<div id="newsct_article" class="newsct_article _article_body">
		<article id="dic_area" class="go_trans _article_content" style="-webkit-tap-highlight-color: rgba(0,0,0,0)">
			<div style="display:box;border-left:solid 4px rgb(228, 228, 228);padding-left: 20px; padding-right: 20px;">35만달러…현재 14만달러보다 3배 이상 뛴다<br>“달러는 가짜 화폐…시장 무너질 것”</div><br><br><table class="nbd_table"><tbody><tr><td><span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP ">
	<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
		<img id="img1" class="_LAZY_LOADING _LAZY_LOADING_INIT_HIDE" alt="" style="" src="https://imgnews.pstatic.net/image/016/2025/01/06/0002412110_001_20250106214215903.jpg?type=w860">
	</div>
</div></span></td></tr><tr><td>로버트 기요사키. [로이터]</td></tr></tbody></table><br><br>[헤럴드경제=문혜현 기자] 베스트셀러 ‘부자 아빠 가난한 아빠’의 저자 로버트 기요사키가 올해 비트코인 가격이 지금보다 세 배 이상 오를 것으로 전망했다.<br><br>5일(현지시간) 야후파이낸스에 따르면 기요사키는 올해 비트코인 목표가를 35만달러(약 5억1400만원)로 올렸다. 이날 기준 14만달러를 웃도는 비트코인이 3배 넘게 오를 것이란 예측이다.<br><br>또한 그는 현재 글로벌 증시에 거품이 껴있고, 곧 붕괴할 것이라고 주장했다. 기요사키는 “가짜 화폐에 해당하는 달러를 계속 뽑고 돌리고 있는 재무부, 연방준비제도(<span data-type="ore" data-lang="en">Fed·</span>연준), 은행 때문에 시장은 무너질 것이고 비트코인·금·은을 보유하고 있는 사람들은 살아남을 것”이라고 말했다.<br><br>기요사키는 지난해 11월 비트코인 가격이 10만달러가 넘으면 모두 팔아야 한다고 주장했지만, 도널드 트럼프 전 미국 대통령이 대선에서 당선되면서 비트코인 가격이 급등하자 다음 달인 12월 자신의 <span data-type="ore" data-lang="en">SNS</span>에서 25만달러까지 오를 것이라고 전망을 바꿨다.<br><br>그는 당시 암호화폐 전문매제 핀볼드를 통해 “살진 돼지는 결국 도축 당한다는 사실을 기억해야 한다”며 “투자자들이 돼지가 되어선 안 된다”며 경고하기도 했다.<br><br>
		</article>
	</div>

In [ ]:
<article id="dic_area" class="go_trans _article_content" style="-webkit-tap-highlight-color: rgba(0,0,0,0)">
			<b><b>경영권 분쟁 장외 여론전 격화<br><br>2022년 3억달러 투입해 회사 인수<br>2021년 설립 시 자본금 275만달러<br>고려아연 “주주간 거래 알 수 없어”<br><br>23일 임시주총 ‘집중투표제’ 현안<br>지분율 낮은 최윤범측 도입 주장<br>영풍 “소수주주 뜻 반영 안돼” 반대<br>4.51% 지분 국민연금 등 선택 중요</b><br></b><br>고려아연 경영권을 놓고 최윤범 회장 측과 <span data-type="ore" data-lang="en">MBK</span>파트너스·영풍 간 분쟁이 임시 주주총회를 앞두고 더욱 치열해지고 있다. 지분 과반을 차지하지 못한 양측은 소액주주들을 끌어들이기 위한 여론전을 강화하는 모습이다.<br>  &nbsp;<br>  <span data-type="ore" data-lang="en">MBK·</span>영풍은 6일 보도자료를 내고 2022년 고려아연이 미국 전자폐기물 재활용업체 이그니오를 인수했을 당시 설립되지 얼마 안 된 이 기업을 터무니없는 밸류에이션(가치평가)을 책정해 매도자들에게 최대 100배라는 수익을 안겨줬다고 주장하고 나섰다.<br>  &nbsp;<br>  고려아연이 당시 인수대금으로 치른 금액은 약 3억달러인데, 2021년에 설립된 이그니오의 초기 출자 자본금 총액은 약 275만달러로 100배 이상의 이익을 거둔 기존 주주의 정체를 모른다는 고려아연 측 주장이 납득가지 않는다고도 했다.<br>  <table class="nbd_table"><tbody><tr><td> <span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP ">
	<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
		<img id="img1" class="_LAZY_LOADING _LAZY_LOADING_INIT_HIDE" alt="" style="" src="https://imgnews.pstatic.net/image/022/2025/01/06/20250106520833_20250106214609672.jpg?type=w860">
	</div>
</div></span></td></tr></tbody></table>  이에 고려아연 측은 인수 당시 구주주를 대표한 회사와 거래했으며 주주 간 거래는 알 수 있는 부분이 아니라고 반박했다.<br>  &nbsp;<br>  고려아연 경영권 분쟁은 지난해 9∼11월 양측 간 주식 공개매수와 고려아연의 유상증자 시도를 둘러싼 ‘1라운드’에 이어 이달 23일 임시 주총을 앞둔 2라운드로 넘어간 형국이다.<br>  &nbsp;<br>  고려아연 이사회의 소집 결의로 개최되는 임시 주총에 올라올 안건 중에서는 집중투표제 도입이 최대 현안이다. 집중투표제는 소액주주 권익을 보호하고자 신규 이사 복수 선임 시 주주가 의결권을 몰아줄 수 있는 제도다. 예를 들어 이사 5명을 선임한다면 1주당 5개의 의결권을 부여하는데, 해당 주주가 의결권 5개를 특정 후보에게 몰아줄 수 있다. 오너 위주의 지배구조를 유지해온 국내 재계는 이 제도 도입에 소극적이었다.<br>  &nbsp;<br>  최 회장 측이 집중투표제 도입을 시도하는 것은 <span data-type="ore" data-lang="en">MBK·</span>영풍에 비해 낮은 지분율 탓으로 분석된다. <span data-type="ore" data-lang="en">MBK·</span>영풍 측은 지난해 말 기준 발행주식 총수의 40.97%, 자기주식을 뺀 의결권 주식 총수 기준으로는 46.7%를 각각 확보했다고 밝힌 바 있다. 발행주식 총수의 33∼34%를 차지한 것으로 짐작되는 최 회장 측을 앞서는 수준이다.<br>  &nbsp;<br>  이에 최 회장은 최근 주주들에게 보낸 서한에서 “고려아연의 현 경영진은 기업의 지배구조는 지속해서 개선돼야 한다고 굳게 믿고 있다”며 “이를 충실히 이행하기 위해 이사회는 임시 주총 안건으로 주주 친화적이며 주주권익 보호에 중점을 둔 의안들을 상정하기로 결정했다”고 집중투표제 도입 이유를 설명했다.<br>  &nbsp;<br>  <span data-type="ore" data-lang="en">MBK·</span>영풍 측은 집중투표제 도입을 해도 소수주주들의 뜻이 관철될 확률이 매우 낮다면서 반대한다. 경영권 분쟁의 여파로 <span data-type="ore" data-lang="en">MBK·</span>영풍 측과 최 회장 측으로 구성된 1·2대 주주가 총 지분의 80∼90%를 쥐고 있는데 이 구도에서는 소수주주들이 선호하는 신규 이사 후보를 선임할 수 없다는 주장이다.<br>  &nbsp;<br>  <span data-type="ore" data-lang="en">MBK·</span>영풍은 “집중투표제 도입 의안이 가결되고 이사진 수가 19인으로 제한되면 주요 주주의 보유 지분을 고려했을 때 집중투표로 이사를 선임할 수 있는 주체는 사실상 1대 및 2대 주주에 한정되고, 기타 소수주주 측 이사 선임은 사실상 어렵다”고 주장했다.<br>  &nbsp;<br>  양측 지분이 절반을 넘지 못한 상황인지라 다른 주주의 선택이 집중투표제 도입 여부에 중요할 변수일 수밖에 없다. 국민연금은 이날 전자공시를 통해 지난해 10월 말 기준 고려아연 주식을 4.51% 보유하고 있다고 밝혔다. 9월 말 기준 7.49%보다 3%포인트가량 비중을 낮춘 것으로 차익 시현에 나선 것으로 풀이된다. 최 회장 측 우호 지분으로 분류되지만, 집중투표제 도입에는 소극적인 다른 재벌그룹의 보유 지분도 변수가 될 수 있다.<br><br>
		</article>